In [468]:
import sys
print(sys.executable)

!{sys.executable} -m pip list

c:\Users\MANEL\anaconda3\python.exe
Package                           Version
--------------------------------- -------------------
aext-assistant                    4.20.0
aext-assistant-server             4.20.0
aext-core                         4.20.0
aext-core-server                  4.20.0
aext_environments_server          4.20.0
aext-panels                       4.20.0
aext-panels-server                4.20.0
aext-project-filebrowser-server   4.20.0
aext-share-notebook               4.20.0
aext-share-notebook-server        4.20.0
aext-shared                       4.20.0
aext-toolbox                      4.20.0
aiobotocore                       2.19.0
aiohappyeyeballs                  2.4.4
aiohttp                           3.11.10
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
alembic                           1.16.5
altair                            5.5.0
anaconda-anon-usage               0.7.1
anaconda-au

In [469]:
import requests
import bs4
import re
import pandas as pd


In [470]:
# on sépare les biens en catégorie (appartement, maison, terrain, etc.)
# on fait une liste afin de les reconnaitre
# sur la page internet, la recherche est différenciée par des sous ensembles
# th#list regroupe les maison
# tf#list regroupe les appartements
# tl#list regroupe les terrains
# tc#list regroupe les commerces
list_bien = ["tf#list", "th#list", "tl#list", "tc#list"]

In [471]:

url_appart = "https://www.etreproprio.com/annonces/tl#list"

request_text = requests.get(
    url_appart,
    headers={"User-Agent": "Python for data science 'appart project'"}
).content

In [472]:
page = bs4.BeautifulSoup(request_text, "lxml")

In [473]:
# Récupérer la class:  <div class="ep-search-list-wrapper"
ep_search_list_wrapper = page.find('div', {'class': 'ep-search-list-wrapper'})

In [474]:
a_list = ep_search_list_wrapper.find_all('a')


In [475]:
href_list = []
for a in a_list:
    href = a.get('href')
    href_list.append(href)

In [476]:
href_list[0]

'https://www.etreproprio.com/immobilier-19641150-vente-terrain-14830m-a-saint-laurent-du-maroni-saint-laurent-du-maroni'

In [477]:
info_bien_dic = {}

In [478]:
# nous savons que les biens que nous allons selectionner sont des terrains
info_bien_dic["type_de_bien"] = "terrain"

In [479]:
# on recupére la page de la première annonce
request_text = requests.get(
    href_list[0],
    headers={"User-Agent": "Python for data science 'appart project'"}
).content
page = bs4.BeautifulSoup(request_text, "lxml")

In [480]:
# Récupérer la class:  ep-area, dtl-main-surface-terrain, ep-price, ep-room
# pour récupérer respectivement la surface, la surface du jardin, le prix et le nombre de pièces
ep_dtl = page.find('div', {'class': 'ep-area'})
ep_dtl_garden = ep_dtl.find('span', {'class': 'dtl-main-surface-terrain'})
ep_price = page.find('div', {'class': 'ep-price'})
ep_room = page.find('div', {'class': 'ep-room'})
ep_loc = page.find('div', {'class': 'ep-loc'})

In [481]:
match = re.search(r"(\d[\d\s\u00A0]*)", ep_dtl.get_text().strip())
surface = match.group().replace("\u00A0", "")
if info_bien_dic["type_de_bien"] == "terrain":
    info_bien_dic["surface_terrain"] =  surface
    info_bien_dic["surface_jardin"] =  None


In [482]:
ep_dtl_garden

In [483]:

if ep_dtl_garden:
    match_garden = re.search(r"\d[\d\xa0]*", ep_dtl_garden.get_text().strip())
    surface_garden = match_garden.group().replace("\xa0", "")
    info_bien_dic["surface_jardin"] =  surface_garden
else:
    info_bien_dic["surface_jardin"] = None

In [484]:
if ep_room:
     match_room = re.search(r"\d+", ep_room.get_text().strip())
     room = match_room.group()
     info_bien_dic["nombre_de_pieces"] =  room
else:
    info_bien_dic["nombre_de_pieces"] =  None

In [485]:
match = re.search(r"(\d[\d\s\u00A0]*)", ep_price.get_text().strip())
price = match.group().replace("\u00A0", "")
info_bien_dic["price"] =  price

In [486]:
match = re.search(r"—\s*(.*?)\s+(\d{5})\s*—", ep_loc.get_text().strip())
if match:
    ville = match.group(1)
    code_postal = match.group(2)
info_bien_dic["ville"] = ville
info_bien_dic["code_postal"] =code_postal

In [487]:
info_bien_dic

{'type_de_bien': 'terrain',
 'surface_terrain': '14 830',
 'surface_jardin': None,
 'nombre_de_pieces': None,
 'price': '577 500',
 'ville': 'Saint-Laurent-Du-Maroni',
 'code_postal': '97320'}

In [488]:
# récupérer le lien pour la page suivante :
class_next_page = page.find('div', {'class': 'ep-nav-next'})
class_next_page

In [489]:
a_next_page = class_next_page.find('a')
href_next_page = a_next_page.get('href')
href_next_page

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:

for i in range(10):

    request = requests.get(
        href_next_page,
        headers={"User-Agent": "Python for data science appart exercice"}
    ).content
    page = bs4.BeautifulSoup(request_text, "lxml")
    ep_search_list_wrapper = page.find('div', {'class': 'ep-search-list-wrapper'})
    a_list = ep_search_list_wrapper.find_all('a')
    for a in a_list:
        href = a.get('href')
        href_list.append(href)
    class_next_page = page.find('div', {'class': 'ep-nav-next'})
    if class_next_page is None:
        break
    else:
        a_next_page = class_next_page.find('a')
        href_next_page = a_next_page.get('href')


    

In [ ]:
len(href_list)

275